In [3297]:
import numpy as np
from PIL import Image, ImageDraw, ImageColor

In [3298]:
class RotationTranslationMatrix:
    def __init__(self, m) -> None:
        assert m.shape[0] == m.shape[1]
        self.m = m

In [3299]:
class Vector:
    def __init__(self, v):
        if v.dtype != np.float16:
            v = v.astype(np.float16)
        self.v = v

    def __eq__(self, __o):
        return np.all(self.v == __o.v)

    def __add__(self, __o):
        return Vector(self.v + __o.v)

    def __sub__(self, __o):
        return Vector(self.v - __o.v)

    def __mul__(self, __o):
        return Vector(self.v * __o)
    
    def to_PointV(self):
        return PointV(v=self.v)

    def norm(self):
        return np.linalg.norm(self.v)

    def unit(self):
        return self.v / self.norm()

    def __repr__(self) -> str:
        return f"Vector {self.v}"

In [3300]:
class PointV:
    def __init__(self, *xyz, v=None):
        if xyz:
            self.v = np.array(xyz, dtype=np.float16) 
        else:
            if v.dtype != np.float16:
                v = v.astype(np.float16)
            self.v = v

    def v_affine(self):
        return self.v[:-1] / self.v[-1]

    def v_homogenious(self):
        return np.append(self.v, 1) 
    
    def __hash__(self) -> int:
        return hash(self.v.tobytes())

    def __eq__(self, __o):
        return np.all(self.v == __o.v)

    def __add__(self, __o):
        if isinstance(__o, PointV):
            return PointV(v=self.v + __o.v)
        return PointV(v=self.v + __o)

    def __sub__(self, __o):
        return PointV(v=self.v - __o.v)

    def __mul__(self, value):
        return PointV(v=self.v * value)
    
    def __truediv__(self, value):
        return PointV(v=self.v / value)
    
    def norm(self):
        return np.linalg.norm(self.v)

    def unit(self):
        return self.v / self.norm()

    def __matmul__(self, m):
        return PointVV(self.v_homogenious() @ m).v_affine()

    def __repr__(self) -> str:
        return f"PointV {self.v}"

In [3301]:
class Edge:
    def __init__(
        self,
        a,
        b,
    ):
        self.a = a
        self.b = b
        self.v = b - a
        self.normal = None

    def __eq__(self, __o):
        return (
            self.a == __o.a and self.b == __o.b or \
                self.a == __o.b and self.b == __o.a
        )
    
    def __hash__(self) -> int:
        a_bytes = self.a.v.tobytes()
        b_bytes = self.b.v.tobytes()
        return hash(tuple(sorted((a_bytes, b_bytes))))

    def normal_to(self, viewPointV):
        v_a = self.a - viewPointV
        v_b = self.b - self.a
        proj = v_b * np.dot(v_a.v, v_b.v) * (1 / v_b.norm() ** 2)
        n = (v_a - proj).unit()
        return n

    def __repr__(self) -> str:
        return f"Edge <{self.a} {self.b}>"

In [ ]:
# TODO
# Polygon -> SimplePolygon -> Triangle

import functools
import operator
from ordered_set import OrderedSet

class Polygon:
    def __init__(self, **kwargs) -> None:

        self.vertices = None
        self.edges = None
        self.rotation = None
        self.L = None
        self.convex_type = None
        self.simple_type = None
        self.signed_area = None
        self.triangluation = []
        self.centroid = None
        self.edge_normals = None
        self.simple_polygon_list = []
        self.color = "black"
        self.__dict__.update(kwargs)

        assert self.vertices or self.edges, "Polygon: vertices and edges are empty"

        if self.vertices is None:
            self._vertices()

        if self.edges is None:
            self._edges()

        if self.L is None:
            self.L = len(self.vertices)

        if self.convex_type is None:
            self._convex_type()

        if self.simple_type is None:
            self._simple_type()

        if self.simple_polygon_list == []:
            if self.simple_type == "SELF_INTERSECTED":
                self._simple_polygon_list()
            else:
                self.simple_polygon_list = [self]

        if self.triangluation == []:
            self._triangluation()
        
        if self.signed_area is None:
            self._signed_area()

        if self.rotation is None:
            self._rotation()
        
        if self.centroid is None:
            self._centroid()

        if self.edge_normals is None:
            self._edge_normals()

        self.color_rgb = ImageColor.getrgb(self.color)

    def _edges(self):
        self.edges = OrderedSet(
            Edge(p1, p2)
            for p1, p2 in zip(self.vertices, self.vertices[1:] + [self.vertices[0]])
        )

    def _vertices(self):
        self.vertices = tuple(edge.a for edge in self.edges)

    def _convex_type(self):
        flag = None
        for i in range(self.L):
            for j in range(i + 1, i + self.L - 1):
                cl_type = classifyEdgePointV(self.edges[j % self.L], self.vertices[i])
                if flag is None:
                    flag = cl_type
                elif flag != cl_type:
                    self.convex_type = "CONCAVE"
                    return
        self.convex_type = "CONVEX"

    def _simple_type(self):
        flag = "SIMPLE"
        intersection_pairs = {}
        for i in range(self.L - 1):
            for j in range(i + 2, self.L):
                if i == (j + 1) % self.L:
                    continue
                vi, vj = self.edges[i], self.edges[j]
                cross_type, ti = crossEdges(vi, vj)
                if cross_type == "SKEW_CROSS":
                    intersection_PointV = vi.a + (vi.b - vi.a) * ti
                    tj = (intersection_PointV - vj.a).norm() / vj.v.norm()
                    intersection_pairs[(i, j)] = (intersection_PointV, ti, tj)
                    flag = "SELF_INTERSECTED"
        self.simple_type = flag
        self._intersection_pairs = intersection_pairs

    def _rotation(self):
        self.rotation = "CCW" if self.signed_area > 0 else "CW"

    def _simple_polygon_list(self):
        # Step 1: Collect intersection PointVs per edge
        splits = {}  # edge_idx -> [(t_param, PointV), ...]
        for (i, j), (PointV, ti, tj) in self._intersection_pairs.items():
            for idx, t in ((i, ti), (j, tj)):
                splits.setdefault(idx, []).append((t, PointV))

        # Sort splits by parameter t for each edge
        for split_list in splits.values():
            split_list.sort(key=lambda x: x[0])

        # Build new edges from sorted splits
        split_edges = {}
        for idx, split_list in splits.items():
            edge = self.edges[idx]
            segments = [edge.a] + [pt for _, pt in split_list] + [edge.b]
            split_edges[idx] = [
                Edge(segments[i], segments[i + 1]) for i in range(len(segments) - 1)
            ]

        stack_edges = [[]]
        for i in range(self.L):
            if i not in split_edges:
                stack_edges[-1].append(self.edges[i])
                if (
                    len(stack_edges[-1]) > 1 and \
                        stack_edges[-1][0].a == stack_edges[-1][-1].b
                ):
                    self.simple_polygon_list.append(
                        Polygon(
                            edges=stack_edges.pop(), 
                            simple_type="SIMPLE",
                            color=self.color
                            )
                        )
            else:
                for edge in split_edges[i]:
                    stack_edges[-1].append(edge)
                    if (
                        len(stack_edges[-1]) > 1 and \
                            stack_edges[-1][0].a == stack_edges[-1][-1].b
                    ):
                        self.simple_polygon_list.append(
                            Polygon(
                                edges=stack_edges.pop(), 
                                simple_type="SIMPLE",
                                color=self.color
                                )
                            )
                    else:
                        stack_edges.append([])
                if stack_edges:
                    del stack_edges[-1]

    def _signed_area(self):
        self.signed_area = 0.5 * sum(np.cross(edge.a.v, edge.b.v) for edge in self.edges)
    
    def _edge_normals(self):
        for edge in self.edges:
            edge.normal = edge.normal_to(viewPointV=self.centroid)

    def EO(self, p):
        intersection_sum = 0
        for edge in self.edges:
            match classifyEdgePointV(edge, p):
                case "TOUCHING":
                    return "INSIDE"
                case "CROSS_LEFT":
                    intersection_sum += 1
                case "CROSS_RIGHT":
                    intersection_sum += 1
        if intersection_sum % 2 == 1:
            return "INSIDE"
        return "OUTSIDE"

    def NZW(self, p):
        winding_number = 0
        for edge in self.edges:
            match classifyEdgePointV(edge, p):
                case "TOUCHING":
                    return "INSIDE"
                case "CROSS_LEFT":
                    winding_number += 1
                case "CROSS_RIGHT":
                    winding_number -= 1
        if winding_number == 0:
            return "OUTSIDE"
        return "INSIDE"
    
    def _centroid(self):
        self.centroid = functools.reduce(operator.add, self.vertices) / self.L
    
    def join(self, other: Polygon):
        assert self.rotation != other.rotation, "Polygons should have different rotations"
        self.edges = self.edges.symmetric_difference(other.edges)
        self._vertices() 

    def _triangluation(self):
        sorted_PointVs = sorted(self.vertices, key=lambda x: x.v[0])
        triangle = Triangle(vertices=sorted_PointVs[:3])
        current = triangle
        self.triangluation.append(current)
        for viewPointV in sorted_PointVs[3:]:
            for edge in current.edges:
                if edge_visible(edge, viewPointV):
                    triangle = Triangle(vertices=[edge.a, edge.b, viewPointV])
                    self.triangluation.append(triangle)
                    print(self.rotation)
                    print(triangle.rotation)
                    current.join(triangle)

In [ ]:
class SimplePolygon(Polygon):
    ...

In [ ]:
class Triangle(SimplePolygon):
    def __init__(self, **kwargs) -> None:

        self.vertices = None
        self.edges = None
        self.rotation = None
        self.L = 3
        self.convex_type = "CONVEX"
        self.simple_type = "SIMPLE"
        self.signed_area = None
        self.triangluation = [self]
        self.centroid = None
        self.edge_normals = None
        self.simple_polygon_list = [self]
        self.color = "black"
        self.__dict__.update(kwargs)

        assert self.vertices or self.edges, "Triangle: vertices and edges are empty"

        if self.edges is None:
            self._edges()

        if self.vertices is None:
            self._vertices()

        if self.centroid is None:
            self._centroid()
        
        if self.signed_area is None:
            self._signed_area()

        if self.rotation is None:
            self._rotation()

        if self.edge_normals is None:
            self._edge_normals()

        self.color_rgb = ImageColor.getrgb(self.color)


In [3304]:
def edge_visible(edge, viewvector):
    dot_product = np.dot(edge.normal, viewvector.v)
    return dot_product <= 0


In [3305]:
def draw_polygon(polygon, draw):
    for edge in polygon.edges:
        draw.PointV(pixelEdge(edge), fill=polygon.color)

In [3306]:
def pixelLine(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    ix = 1 if dx > 0 else -1
    iy = 1 if dy > 0 else -1
    dx *= ix
    dy *= iy
    x = x1
    y = y1
    pixels = []
    if dx < dy:
        E = 2 * dx - dy
        for _ in range(dy + 1):
            pixels.append((x, y))
            if E >= 0:
                x += ix
                E -= 2 * dy
            y += iy
            E += 2 * dx
    else:
        E = 2 * dy - dx
        for _ in range(dx + 1):
            pixels.append((x, y))
            if E >= 0:
                y += iy
                E -= 2 * dx
            x += ix
            E += 2 * dy
    return pixels


def pixelEdge(e):
    x1, y1, x2, y2 = (
        round(e.a.v[0]),
        round(e.a.v[1]),
        round(e.b.v[0]),
        round(e.b.v[1]),
    )
    return pixelLine(x1, y1, x2, y2)

In [3307]:
def classify(x1, y1, x2, y2, x, y):
    ax = x2 - x1
    ay = y2 - y1
    bx = x - x1
    by = y - y1
    s = ax * by - ay * bx
    if s > 0:
        return "LEFT"
    if s < 0:
        return "RIGHT"
    if ax * bx < 0 or ay * by < 0:
        return "BEHIND"
    if ax**2 + ay**2 < bx**2 + by**2:
        return "INFRONT"
    if x == x1 and y == y1:
        return "ORIGIN"
    if x == x2 and y == y2:
        return "DESTINATION"
    return "BETWEEN"


def classifyEdgePointV(e, p):
    print(e, p)
    return classify(*e.a.v, *e.b.v, *p.v)

In [3308]:
def intersect(ax, ay, bx, by, cx, cy, dx, dy):
    nx = dy - cy
    ny = cx - dx
    denom = nx * (bx - ax) + ny * (by - ay)
    if denom == 0:
        type = classify(cx, cy, dx, dy, ax, ay)
        if type == "LEFT" or type == "RIGHT":
            return "PARALLEL", None
        else:
            return "SAME", None
    num = nx * (ax - cx) + ny * (ay - cy)
    t = -num / denom
    return "SKEW", t


def intersectEdges(e1, e2):
    return intersect(*e1.a.v, *e1.b.v, *e2.a.v, *e2.b.v)

In [3309]:
def cross(ax, ay, bx, by, cx, cy, dx, dy):
    type, tab = intersect(ax, ay, bx, by, cx, cy, dx, dy)
    if type == "SAME" or type == "PARALLEL" or tab is None:
        return type, None
    if tab < 0 or tab > 1:
        return "SKEW_NO_CROSS", None
    _, tcd = intersect(cx, cy, dx, dy, ax, ay, bx, by)
    if tcd is not None and (tcd < 0 or tcd > 1):
        return "SKEW_NO_CROSS", None
    return "SKEW_CROSS", tab


def crossEdges(e1, e2):
    return cross(*e1.a.v, *e1.b.v, *e2.a.v, *e2.b.v)

In [3310]:
img = Image.new("RGB", (200, 200), "white")
draw = ImageDraw.Draw(img)

In [3311]:
p0 = Polygon(
    vertices=[
        PointV(150, 80),
        PointV(10, 60),
        PointV(50, 100),
        PointV(100, 20),
        PointV(120, 160),
        PointV(190, 100),
        PointV(180, 190),
    ],
    color="blue",
)

Edge <PointV [10. 60.] PointV [ 50. 100.]> PointV [150.  80.]
Edge <PointV [ 50. 100.] PointV [100.  20.]> PointV [150.  80.]
vi.a  PointV [150.  80.]
vi.b  PointV [10. 60.]
ti  0.5737
intersection_PointV  PointV [69.7 68.5]
vi.a  PointV [150.  80.]
vi.b  PointV [10. 60.]
ti  0.302
intersection_PointV  PointV [107.75  73.94]
vi.a  PointV [120. 160.]
vi.b  PointV [190. 100.]
ti  0.5996
intersection_PointV  PointV [162. 124.]
Edge <PointV [10. 60.] PointV [ 50. 100.]> PointV [69.7 68.5]
Edge <PointV [ 50. 100.] PointV [69.7 68.5]> PointV [10. 60.]
Edge <PointV [69.7 68.5] PointV [10. 60.]> PointV [ 50. 100.]
Edge <PointV [69.7 68.5] PointV [100.  20.]> PointV [107.75  73.94]
Edge <PointV [100.  20.] PointV [107.75  73.94]> PointV [69.7 68.5]
Edge <PointV [107.75  73.94] PointV [69.7 68.5]> PointV [100.  20.]
Edge <PointV [190. 100.] PointV [180. 190.]> PointV [162. 124.]
Edge <PointV [180. 190.] PointV [162. 124.]> PointV [190. 100.]
Edge <PointV [162. 124.] PointV [190. 100.]> PointV [1

C:\Users\1\AppData\Local\Temp\ipykernel_12012\4256152422.py:31: RuntimeWarning: overflow encountered in multiply
  return PointV(v=self.v * value)
C:\Users\1\AppData\Local\Temp\ipykernel_12012\4256152422.py:40: RuntimeWarning: invalid value encountered in divide
  return self.v / self.norm()


AssertionError: Polygons should have different rotations

In [ ]:
p0.convex_type

'CONCAVE'

In [ ]:
p0.simple_type

'SELF_INTERSECTED'

In [ ]:
p0.simple_polygon_list

In [ ]:
p0.signed_area

-1800.0

In [ ]:
p0.rotation

'CW'

In [ ]:
draw_polygon(p0, draw)

In [ ]:
p1 = Polygon(
    vertices=[
        PointV(10, 10),
        PointV(40, 100),
        PointV(100, 150),
        PointV(50, 180),
        PointV(70, 10),
    ],
    color="green",
)

In [ ]:
p1.convex_type

'CONCAVE'

In [ ]:
p1.simple_type

'SELF_INTERSECTED'

In [ ]:
p1.simple_polygon_list

[<__main__.Polygon at 0x160b5a1e090>, <__main__.Polygon at 0x160b61764b0>]

In [ ]:
# draw_polygon(p1, draw)

In [ ]:
p2 = Polygon(
    vertices=[
        PointV(20, 10),
        PointV(50, 10),
        PointV(80, 20),
        PointV(90, 50),
        PointV(70, 80),
        PointV(40, 90),
        PointV(10, 60),
    ],
    color="yellow",
)

In [ ]:
p2.convex_type

'CONVEX'

In [ ]:
p2.simple_type

'SIMPLE'

In [ ]:
p2.simple_polygon_list

In [ ]:
# draw_polygon(p2, draw)
# img.show()

In [ ]:
triangle = Polygon(
    vertices=[
        PointV(34, 80),
        PointV(57, 1),
        PointV(170, 80),
    ],
    color="yellow",  # A B  # B C  # C A
)

In [ ]:
def paint_triangle():
    edges = [pixelEdge(edge) for edge in triangle.edges]
    ABC = edges[0] + edges[1]
    i = 0
    AC = edges[2][::-1]
    j = 0
    A = AC[0]
    C = AC[-1]
    dxAC = abs(A[0] - C[0])
    dyAC = abs(A[1] - C[1])
    k = 0 if dxAC > dyAC else 1
    pixels = []
    while ABC[i] != C:
        line = pixelLine(*ABC[i], *AC[j])
        pixels.extend(line)
        i += 1
        if AC[j][k] != ABC[i][k]:
            j += 1
    return pixels

In [ ]:
# draw.PointV(paint_triangle(), fill="red")


img.show()